In [1]:
#Configuração do container e keys
STORAGE_ACCOUNT_NAME = 'nomedastorageaccount'
CONTAINER_INPUT = 'nomecontainerinput'
CONTAINER_OUTPUT = 'nomecontaineroutput'
KEY = ''

#Mount do diretório
if not any(mount.mountPoint == '/mnt/input' for mount in dbutils.fs.mounts()):
  dbutils.fs.mount(
     source = "wasbs://{}@{}.blob.core.windows.net".format(CONTAINER_INPUT, STORAGE_ACCOUNT_NAME),
     mount_point = "/mnt/{}".format(CONTAINER_INPUT),
     extra_configs = {"fs.azure.account.key.{}.blob.core.windows.net".format(STORAGE_ACCOUNT_NAME):"{}".format(KEY)})

In [2]:
FILENAME = "/pasta/arquivo.csv".format(CONTAINER_INPUT)

df = (spark
  .read
  .option("header", True)
  .option("header", True) \
  .option("sep", ',') \
  .csv(FILENAME)
)

In [3]:
#Importando as bibliotecas
from __future__ import print_function
import requests
import json
import pandas as pd
import numpy as np
import warnings
import matplotlib
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.resources import CDN
from bokeh.embed import components, file_html
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

In [4]:
#CONFIGURANDO API ANOMALIA
# To start sending requests to the Anomaly Detector API, paste your subscription key you received after creating Anomaly Detector resource. 
subscription_key = 'subscriptionkey' 

# Use the endpoint your received from overview section of the Anomaly Detector resource you created
# the endpoint is like https://westus2.api.cognitive.microsoft.com/, different by regions, you need to concat anomalydetector/v1.0/timeseries/entire/detect

endpoint = 'endpoint'

In [5]:
#Função que chama a API
def detect(endpoint, subscription_key, request_data):
    #print(request_data)
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [6]:
#Função que trata o retorno da API
def build_figure(sample_data, sensitivity):
    sample_data['sensitivity'] = sensitivity
    result = detect(endpoint, subscription_key, sample_data)    
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']], 
          'value': [x['value'] for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Batch Anomaly Detection ({0} Sensitvity)".format(sensitivity), width=700, height=600)
    for anom in response['isAnomaly']:
       
        if anom == True:
          current_value = int(values[index])
          #print (current_value)
          iloc_expvalue = response.iloc[index]['expectedValues']
          iloc_upperMargins = response.iloc[index]['upperMargins']
          iloc_lowerMargins = response.iloc[index]['lowerMargins']
          sum_exp_upper = iloc_expvalue + iloc_upperMargins
          dif_exp_lower = iloc_expvalue - iloc_lowerMargins
          current_value_greaterthan_sum_exp_upper = current_value > sum_exp_upper
          current_value_lowerthan_sum_exp_upper = current_value < dif_exp_lower
          
          if (current_value_greaterthan_sum_exp_upper or current_value_lowerthan_sum_exp_upper):           
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        
        index = index+1
        
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend_label='Boundary')
    p.line(label, values, legend_label='Value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend_label='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    # create an html document that embeds the Bokeh plot
    html = file_html(p, CDN, "my plot1")
    # display this html
    displayHTML(html)

In [7]:
from pyspark.sql.functions import col, udf, date_format, asc
from pyspark.sql.types import DateType, TimestampType 
from datetime import datetime

#Transformação da data para o valor que a API pecisa receber
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), TimestampType())
df_dt = df.withColumn('timestamp', func(col('TimeStampScript')))

#yyyy-MM-dd'T'HH:mm:ss'Z'
df_dt = df_dt.withColumn("timestamp", date_format(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss'Z'"))

#criação coluna value
df_dt = df_dt.withColumn('value', col('IOOtherBytesPersec'))

In [8]:
#Filtrando pela coluna Outlook do csv
filter = df_dt["Name"]=="OUTLOOK"

#Criando dicionário de dados chave e valor
#df_dict = df_dt.select('PercentProcessorTime', F.to_date(df.TimeStampScript, 'yyyy-MM-ddTHH:mm:ssZ')).where(filter).rdd.map(lambda row: row.asDict()).collect()
df_dict = df_dt.select('value', df_dt.timestamp).where(filter).sort(asc("timestamp")).rdd.map(lambda row: row.asDict()).collect()

dict_anomaly = {}
dict_anomaly["series"] = df_dict 
dict_anomaly['granularity'] = 'minutely'

In [9]:
# Minutely Sample
sample_data = dict_anomaly
sample_data['granularity'] = 'minutely'
#sample_data['period'] = 7
sample_data['customInterval'] = 6

# 85 sensitivity
build_figure(sample_data,90)

<!DOCTYPE html>
 
 
 
 
 
 my plot1

In [10]:
#Resultado API sem o gráfico

sample_data['sensitivity'] = 85
result = detect(endpoint, subscription_key, sample_data)
result

Out[10]: {'expectedValues': [33007369.0,
 33043336.0,
 33077944.666666668,
 34472029.75,
 35412966.8,
 36653421.6,
 37886013.4,
 39111411.2,
 39246982.2,
 39308702.0,
 39377317.4,
 39452869.6,
 39599695.8,
 39781653.6,
 39966852.0,
 40164542.2,
 40370503.4,
 40651031.8,
 40963379.0,
 41318087.2,
 41903215.6,
 42483096.0,
 42926402.8,
 43486743.4,
 44027160.4,
 44484217.8,
 45506563.0,
 46700457.2,
 47933290.6,
 47781465.06666665,
 47509698.76666665,
 46656161.06666665,
 45674692.00512819,
 41722022.20512819,
 39447822.93846154],
 'isAnomaly': [False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False],
 'isNegativeAnomaly': [False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False],
 'isPositiveAnomaly': [False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False],
 'lowerMargins': [105418.62000000104,
 36326.67,
 105418.62000000104,
 4224077.8025,
 3801385.682000003,
 2611787.264013119,
 1369811.0860000015,
 164367.19799999698,
 86009.37799999698,
 178378.12,
 177156.6260000015,
 168824.9339999985,
 412672.062000003,
 1284563.734482132,
 802056.4761575684,
 549816.762097165,
 282118.8560000015,
 767491.122000003,
 848688.86,
 1228200.717117153,
 1579233.3739999984,
 1076796.35,
 2577605.9262966886,
 1129810.8460000015,
 7630472.7626895085,
 1280561.022000003,
 3185937.94,
 2925436.517999997,
 2864606.8439999986,
 3229318.3826666856,
 5182569.805666682,
 946936.9400000051,
 2239594.800769232,
 10977962.205128193,
 6757925.087846158],
 'period': 0,
 'upperMargins': [105418.62000000104,
 36326.67,
 105418.62000000104,
 4224077.8025,
 3801385.682000003,
 2611787.264013119,
 1369811.0860000015,
 164367.19799999698,
 86009.37799999698,
 178378.12,
 177156.6260000015,
 168824.9339999985,
 412672.062000003,
 1284563.734482132,
 802056.4761575684,
 549816.762097165,
 282118.8560000015,
 767491.122000003,
 848688.86,
 1228200.717117153,
 1579233.3739999984,
 1076796.35,
 2577605.9262966886,
 1129810.8460000015,
 7630472.7626895085,
 1280561.022000003,
 3185937.94,
 2925436.517999997,
 2864606.8439999986,
 3229318.3826666856,
 5182569.805666682,
 946936.9400000051,
 2239594.800769232,
 11247051.794871807,
 6757925.087846158]}